In [1]:
## DO NOT ERASE THIS. IMPORTANT TO CORRECTLY IMPORT MODULES
import sys

sys.path.append("../")
sys.executable

'/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/bin/python3.12'

In [2]:
import pandas as pd

pd.set_option("display.max_rows", 102)

from tqdm.auto import tqdm
import mmh3

import torch
from torch.utils.data import DataLoader, TensorDataset

from src.utils import (
    load_MovieLens,
    create_test_user,
    train_test_val_split,
    seed_everything,
    trainDatasetWithCrossFeatures,
    split_test_df,
    add_not_watched_movies,
    create_test_user_display_df,
    train,
    predict,
)
from src.models import wideAndDeep
from src.metrics import reccomendation_report

## Constants

In [3]:
DATA_FOLDER = "../../data/ml-1m/"
RANDOM_STATE = 7

In [4]:
seed_everything(RANDOM_STATE)

## Data

In [5]:
df_users, df_movies, df_ratings = load_MovieLens(DATA_FOLDER)

### Creating a test_user

In [6]:
df_users, df_ratings, new_user_id = create_test_user(
    df_users, df_ratings, [6, 16, 3192, 1461, 827, 887, 593]
)
print("Test user watch list:")
display(df_ratings[df_ratings.userId == new_user_id].merge(df_movies, on="movieId"))

Test user watch list:


,userId,movieId,rating,timestamp,name,genre
0,6040,6,5,0,Sabrina (1995),Comedy|Romance
1,6040,16,5,1,Sense and Sensibility (1995),Drama|Romance
2,6040,3192,5,2,Singles (1992),Comedy|Drama|Romance
3,6040,1461,5,3,Love and Other Catastrophes (1996),Romance
4,6040,827,5,4,Emma (1996),Comedy|Drama|Romance
5,6040,887,5,5,Singin' in the Rain (1952),Musical|Romance
6,6040,593,5,6,Pretty Woman (1990),Comedy|Romance


### Train-test split
Methodology: Last user interaction is a test item. The rest is train. Validation part is 20% of test.

In [7]:
df_train, df_test, df_val = train_test_val_split(df_ratings, df_movies, RANDOM_STATE)

Enriching test:   0%|          | 0/4832 [00:00<?, ?it/s]

Enriching val:   0%|          | 0/1209 [00:00<?, ?it/s]

### Loading additional data

In [8]:
# cos_dist = computeCosineSimilarities(df_train, "userId", "movieId", df_movies["movieId"].nunique())

# with open("../../data/cos_dist.pt", "wb") as f:
#     torch.save(cos_dist, f)

with open("../../data/cos_dist.pt", "rb") as f:
    cos_dist = torch.load(f)
popularity = torch.bincount(
    torch.tensor(df_train.movieId.values), minlength=df_movies["movieId"].nunique()
)
popularity = popularity / torch.max(popularity)

### Building cross features

In [9]:
cross_feats_dim = 30
df_users["combined_feat"] = (
    df_users["gender"].astype(str)
    + df_users["age"].astype(str)
    + df_users["occupation"].astype(str)
)
user_features_mapping = {
    i: df_users.loc[df_users.userId == i, "combined_feat"].values[0]
    for i in df_users.userId.unique()
}
movie_features_mapping = {
    i: df_movies.loc[df_movies.movieId == i, "genre"].values[0]
    for i in df_movies.movieId.unique()
}


# cross features for test
test_idx = (
    (
        pd.Series(list(map(lambda x: user_features_mapping[x], df_test["userId"])))
        + pd.Series(list(map(lambda x: movie_features_mapping[x], df_test["movieId"])))
    )
    .apply(lambda x: mmh3.hash(x) % cross_feats_dim)
    .values
)
cross_test = torch.zeros(df_test.shape[0], cross_feats_dim)
cross_test[torch.arange(cross_test.shape[0]), test_idx] = 1


# cross features for val
val_idx = (
    (
        pd.Series(list(map(lambda x: user_features_mapping[x], df_val["userId"])))
        + pd.Series(list(map(lambda x: movie_features_mapping[x], df_val["movieId"])))
    )
    .apply(lambda x: mmh3.hash(x) % cross_feats_dim)
    .values
)
cross_val = torch.zeros(df_val.shape[0], cross_feats_dim)
cross_val[torch.arange(cross_val.shape[0]), val_idx] = 1

## Wide-and-Deep

In [10]:
seed_everything(RANDOM_STATE)
train_loader = DataLoader(
    trainDatasetWithCrossFeatures(
        df_train,
        df_movies["movieId"].nunique(),
        user_features_mapping,
        movie_features_mapping,
        hash_bucket_size=cross_feats_dim,
    ),
    batch_size=2048,
    shuffle=True,
)
val_loader = DataLoader(
    TensorDataset(
        torch.tensor(df_val["userId"]), torch.tensor(df_val["movieId"]), cross_val
    ),
    batch_size=4096,
    shuffle=False,
)
test_loader = DataLoader(
    TensorDataset(
        torch.tensor(df_test["userId"]), torch.tensor(df_test["movieId"]), cross_test
    ),
    batch_size=4096,
    shuffle=False,
)
num_users = df_users["userId"].nunique()
num_items = df_movies["movieId"].nunique()

  0%|          | 0/994175 [00:00<?, ?it/s]

  0%|          | 0/4970875 [00:00<?, ?it/s]

In [11]:
seed_everything(RANDOM_STATE)
wd = wideAndDeep(
    num_users,
    num_items,
    cross_feats_dim=cross_feats_dim,
    n_mlp_layers=4,
    mlp_layers_dim=32,
    mlp_kwargs={
        "activation": True,
        "dropout": True,
        "batchnorm": True,
        "dropout_rate": 0.6,
    },
)
display(wd)

wideAndDeep(
  (user_embedding): Embedding(6041, 16)
  (item_embedding): Embedding(3883, 16)
  (mlp): Sequential(
    (MLP_layer_0): MLP(
      (block): Sequential(
        (Linear): Linear(in_features=32, out_features=32, bias=True)
        (Activation): ReLU()
        (Dropout): Dropout(p=0.6, inplace=False)
        (BatchNorm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (MLP_layer_1): MLP(
      (block): Sequential(
        (Linear): Linear(in_features=32, out_features=32, bias=True)
        (Activation): ReLU()
        (Dropout): Dropout(p=0.6, inplace=False)
        (BatchNorm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (MLP_layer_2): MLP(
      (block): Sequential(
        (Linear): Linear(in_features=32, out_features=32, bias=True)
        (Activation): ReLU()
        (Dropout): Dropout(p=0.6, inplace=False)
        (BatchNorm): BatchNorm1d(32, eps=1e-05, momentum=0.1, 

In [12]:
optimizer = torch.optim.Adam(wd.parameters(), lr=3e-3)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)
criterion = torch.nn.BCELoss()
n_epochs = 10
device = "cpu"

In [13]:
%%time
train(
    wd,
    train_loader,
    optimizer,
    scheduler,
    criterion,
    n_epochs,
    val_loader=val_loader,
    df_val=df_val,
    cos_dist=cos_dist,
    popularity=popularity,
    verbose=True,
)

Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/2428 [00:00<?, ?batch/s]

Epoch: 0
Train loss: 0.41626


Inference:   0%|          | 0/30 [00:00<?, ?batch/s]

{'Hit rate @ 15': tensor(0.5641),
 'NDCG @ 15': tensor(0.2395),
 'Diversity (ILD)': tensor(0.1430),
 'Novelty (EPC)': tensor(0.7244)}


...

{'Hit rate @ 15': tensor(0.5798),
 'NDCG @ 15': tensor(0.2715),
 'Diversity (ILD)': tensor(0.1448),
 'Novelty (EPC)': tensor(0.7007)}


  0%|          | 0/2428 [00:00<?, ?batch/s]

Epoch: 9
Train loss: 0.36896


Inference:   0%|          | 0/30 [00:00<?, ?batch/s]

{'Hit rate @ 15': tensor(0.5806),
 'NDCG @ 15': tensor(0.2698),
 'Diversity (ILD)': tensor(0.1447),
 'Novelty (EPC)': tensor(0.7017)}
CPU times: user 26min 25s, sys: 1h 15min 40s, total: 1h 42min 6s
Wall time: 12min


In [14]:
df_test["pred"] = predict(wd, test_loader, verbose=False).numpy()
pred, target, pred_items = split_test_df(df_test, "userId", "movieId", "pred", "action")
reccomendation_report(pred, target, pred_items, cos_dist, popularity, k=15)

{'Hit rate @ 15': tensor(0.5604),
 'NDCG @ 15': tensor(0.2679),
 'Diversity (ILD)': tensor(0.1454),
 'Novelty (EPC)': tensor(0.6998)}

## Test user recommendations

In [15]:
df_test_user = add_not_watched_movies(new_user_id, df_test, df_train, df_movies)

# cross features for test user
test_idx = (
    (
        pd.Series(list(map(lambda x: user_features_mapping[x], df_test_user["userId"])))
        + pd.Series(
            list(map(lambda x: movie_features_mapping[x], df_test_user["movieId"]))
        )
    )
    .apply(lambda x: mmh3.hash(x) % cross_feats_dim)
    .values
)
cross_test_user = torch.zeros(df_test_user.shape[0], cross_feats_dim)
cross_test_user[torch.arange(cross_test_user.shape[0]), test_idx] = 1

test_user_loader = DataLoader(
    TensorDataset(
        torch.tensor(df_test_user["userId"]),
        torch.tensor(df_test_user["movieId"]),
        cross_test_user,
    ),
    batch_size=4096,
    shuffle=False,
)

In [16]:
df_test_user["pred"] = predict(wd, test_user_loader, verbose=False).numpy()
display(create_test_user_display_df(df_test_user, df_movies, "pred"))

,userId,movieId,name,genre,pred
0,6040,1227,"Graduate, The (1967)",Drama|Romance,0.621546
1,6040,2222,Edward Scissorhands (1990),Drama|Romance,0.621535
2,6040,523,Schindler's List (1993),Drama|War,0.610455
3,6040,1203,"Godfather: Part II, The (1974)",Action|Crime|Drama,0.610390
4,6040,2588,"Rocky Horror Picture Show, The (1975)",Comedy|Horror|Musical|Sci-Fi,0.609445
5,6040,1656,Good Will Hunting (1997),Drama,0.609445
6,6040,1081,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi,0.606774
7,6040,2220,"Player, The (1992)",Comedy|Drama,0.606644
8,6040,1220,"Terminator, The (1984)",Action|Sci-Fi|Thriller,0.606589
9,6040,1078,"Crying Game, The (1992)",Drama|Romance|War,0.605579
